# DeepSeek-V3


[DeepSeek-V3](https://github.com/deepseek-ai/DeepSeek-V3) is a strong Mixture-of-Experts (MoE) language model that delivers exceptional performance and speed, surpassing its predecessors. With its advanced capabilities, DeepSeek-V3 ranks among the top open-source models and matches the performance of leading closed-source models.

In this notebook, we will demonstrate how to use the DeepSeek-V3 model for AgentChat within the AG2 framework.

## Requirements

To get started, ensure you meet the following requirements:

1. Install `pyautogen` package:
    - Run the following command to install the [pyautogen](https://pypi.org/project/pyautogen/) package:
    ```bash
    pip install pyautogen
    ```

2. Obtain a DeepSeek API Key:
    - Sign up for a DeepSeek account [here](https://platform.deepseek.com/sign_in). to generate your API key.
    - Refer to the [official documentation](https://api-docs.deepseek.com/) for more information about obtaining and using the API key.

## Set the config for the DeepSeek API

Next, you need to configure the `OAI_CONFIG_LIST` to grant the agent access to the DeepSeek-V3 model.

Here’s an example configuration:

```python
[
    {
        "model": "deepseek-chat",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": "your DeepSeek Key goes here",
        "api_type": "deepseek",
        "tags": ["deepseek"]
    }
]
```

## API parameters

You can include additional parameters in your configuration that are supported by the DeepSeek API. For a full list of parameters and their default values, refer to the [DeepSeek API documentation](https://api-docs.deepseek.com/api/create-chat-completion).

Some examples of such parameters include:

    - temperature (number 0..1)
    - top_p (number 0..1)
    - max_tokens (null, integer >= 0)

Example Configuration:

```python
[

    {
        "model": "deepseek-chat",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": "your DeepSeek Key goes here",
        "api_type": "deepseek",
        "temperature": 0.5,
        "top_p": 0.2, # Note: It is recommended to set temperature or top_p but not both.
        "max_tokens": 10000,
        "tags": ["deepseek"]
    }
]
```


## Two-Agent Coding Example

In this example, we run a two-agent chat with an AssistantAgent (primarily a coding agent) to generate code to count the number of prime numbers between 1 and 10,000 and then it will be executed.

We'll use DeepSeek-V3 in our example

In [1]:
import os

config_list = [
    {
        "model": "deepseek-chat",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": os.environ.get("DEEPSEEK_API_KEY"),
        "api_type": "deepseek",
        "tags": ["deepseek"],
    }
]

Importantly, we have tweaked the system message so that the model doesn't return the termination keyword, which we've changed to FINISH, with the code block.

In [2]:
from pathlib import Path

from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor

# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# Setting up the agents

# The UserProxyAgent will execute the code that the AssistantAgent provides
user_proxy_agent = UserProxyAgent(
    name="User",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: "FINISH" in msg.get("content"),
)

system_message = """You are a helpful AI assistant who writes code and the user executes it.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) for the user to execute.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""

# The AssistantAgent, using DeepSeek-V3 model, will take the coding request and return code
assistant_agent = AssistantAgent(
    name="DeepSeek_Assistant",
    system_message=system_message,
    llm_config={"config_list": config_list},
)

In [3]:
# Start the chat, with the UserProxyAgent asking the AssistantAgent the message
chat_result = user_proxy_agent.initiate_chat(
    assistant_agent,
    message="Provide code to count the number of prime numbers from 1 to 10000.",
)

User (to DeepSeek_Assistant):

Provide code to count the number of prime numbers from 1 to 10000.

--------------------------------------------------------------------------------
DeepSeek_Assistant (to User):

To count the number of prime numbers from 1 to 10000, we can use the Sieve of Eratosthenes algorithm, which is an efficient way to find all primes less than a given number. Here's the Python code to achieve this:

```python
def count_primes(n):
    # Create a boolean array "prime[0..n]" and initialize
    # all entries it as true. A value in prime[i] will
    # finally be false if i is Not a prime, else true.
    prime = [True for _ in range(n+1)]
    p = 2
    while (p * p <= n):
        # If prime[p] is not changed, then it is a prime
        if prime[p] == True:
            # Updating all multiples of p to not prime
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1

    # Counting prime numbers
    prime_count = 0
    for p in range(2,

## Tool Call Example

In this example, instead of writing code, we will show how we can use DeepSeek-V3 model to perform parallel tool calling, where it recommends calling more than one tool at a time.

We’ll use a simple travel agent assistant program where we have a couple of tools for weather and currency conversion.

We start by importing libraries and setting up our configuration to use DeepSeek-V3 model.

In [4]:
import json
import os
from typing import Literal

from typing_extensions import Annotated

import autogen

config_list = [
    {
        "model": "deepseek-chat",
        "base_url": "https://api.deepseek.com/v1",
        "api_key": os.environ.get("DEEPSEEK_API_KEY"),
        "api_type": "deepseek",
        "tags": ["deepseek"],
    }
]

Create our two agents.

In [5]:
# Create the agent for tool calling
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="""For currency exchange and weather forecasting tasks,
        only use the functions you have been provided with.
        Output 'HAVE FUN!' when an answer has been provided.""",
    llm_config={"config_list": config_list},
)

# Note that we have changed the termination string to be "HAVE FUN!"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and "HAVE FUN!" in x.get("content", ""),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
)

Create the two functions, annotating them so that those descriptions can be passed through to the LLM.

We associate them with the agents using `register_for_execution` for the user_proxy so it can execute the function and `register_for_llm` for the chatbot (powered by the LLM) so it can pass the function definitions to the LLM.

In [6]:
# Currency Exchange function

CurrencySymbol = Literal["USD", "EUR"]

# Define our function that we expect to call


def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")


# Register the function with the agent


@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base_amount: Annotated[float, "Amount of currency in base_currency"],
    base_currency: Annotated[CurrencySymbol, "Base currency"] = "USD",
    quote_currency: Annotated[CurrencySymbol, "Quote currency"] = "EUR",
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{format(quote_amount, '.2f')} {quote_currency}"


# Weather function


# Example function to make available to model
def get_current_weather(location, unit="fahrenheit"):
    """Get the weather for some location"""
    if "chicago" in location.lower():
        return json.dumps({"location": "Chicago", "temperature": "13", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "55", "unit": unit})
    elif "new york" in location.lower():
        return json.dumps({"location": "New York", "temperature": "11", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


# Register the function with the agent


@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Weather forecast for US cities.")
def weather_forecast(
    location: Annotated[str, "City name"],
) -> str:
    weather_details = get_current_weather(location=location)
    weather = json.loads(weather_details)
    return f"{weather['location']} will be {weather['temperature']} degrees {weather['unit']}"

We pass through our customers message and run the chat.

In [7]:
# start the conversation
res = user_proxy.initiate_chat(
    chatbot,
    message="What's the weather in New York and can you tell me how much is 123.45 EUR in USD so I can spend it on my holiday? Throw a few holiday tips in as well.",
)

user_proxy (to chatbot):

What's the weather in New York and can you tell me how much is 123.45 EUR in USD so I can spend it on my holiday? Throw a few holiday tips in as well.

--------------------------------------------------------------------------------
chatbot (to user_proxy):


***** Suggested tool call (call_0_5c4b1925-5eec-4190-9887-fecf4dcc4c07): weather_forecast *****
Arguments: 
{"location": "New York"}
***********************************************************************************************
***** Suggested tool call (call_1_f75a7688-2c44-4a81-b8dd-c32b71f5750d): currency_calculator *****
Arguments: 
{"base_amount": 123.45, "base_currency": "EUR", "quote_currency": "USD"}
**************************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION weather_forecast...
Call ID: call_0_5c4b1925-5eec-4190-9887-fecf4dcc4c07
Input arguments: {

We can see that DeepSeek-V3 recommended invoking both tools with the appropriate parameters. The user_proxy executed these calls, and the results were passed back to DeepSeek-V3 for interpretation and response.